# Phase7: Final Dataset Generation - Smart Parking IoT System

## Overview
This notebook generates the final comprehensive dataset combining actual observations and model predictions with all required columns.

### Output Columns:
1. **timestamp** - DateTime of observation/prediction
2. **record_type** - actual/forecast/model_performance
3. **segment_id** - Parking segment identifier
4. **capacity** - Total parking spaces
5. **occupied** - Actually occupied spaces (actual values for comparison)
6. **occupancy_rate** - Occupied/Capacity ratio (actual values for comparison)
7. **available_spots** - Capacity - Occupied (actual values for comparison)
8. **model_source** - Which model generated forecast
9. **predicted_occupied** - Predicted occupied spaces
10. **predicted_occupancy_rate** - Predicted occupancy ratio
11. **mae** - Individual Mean Absolute Error (forecast records only)
12. **rmse** - Individual Root Mean Squared Error (forecast records only)
13. **mape** - Individual Mean Absolute Percentage Error (forecast records only)
14. **hour_of_day** - Hour extracted from timestamp (0-23)
15. **day_of_week** - Day of week (0=Monday to 6=Sunday)
16. **day_name** - Name of day
17. **is_weekend** - Weekend flag (0=weekday, 1=weekend)
18. **is_peak_hour** - Peak hour flag (1=peak 7-9am & 5-7pm, 0=off-peak)
19. **data_split** - Train/Test identification (train/test)

### Key Features:
- **Individual Error Metrics**: Each forecast record now has MAE, RMSE, MAPE
- **Actual vs Predicted**: Forecast records include both actual and predicted values
- **Train/Test Split**: All records are labeled for proper model evaluation
- **Multiple Models**: LSTM (real inference) + Baseline models (synthetic forecasts)

In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import pickle
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [6]:
# Load all necessary data and models
print("🔄 Loading data and models...")

# Define paths
data_path = Path(r"C:\Users\vedp3\OneDrive\Desktop\AAI_530_Final_Project\AAI530-Group10-smart-parking-iot-forecasting\data\raw\smart_parking_full.csv")
models_dir = Path(r"C:\Users\vedp3\OneDrive\Desktop\AAI_530_Final_Project\AAI530-Group10-smart-parking-iot-forecasting\models")
inference_dir = Path(r"C:\Users\vedp3\OneDrive\Desktop\AAI_530_Final_Project\AAI530-Group10-smart-parking-iot-forecasting\data\inference")

# Load original dataset
df_original = pd.read_csv(data_path, sep=';')
df_original['timestamp'] = pd.to_datetime(df_original['timestamp'])
print(f"✅ Original dataset loaded: {df_original.shape}")

# Load LSTM inference results
lstm_results_path = inference_dir / "lstm_inference_complete_results.csv"
if lstm_results_path.exists():
    df_lstm = pd.read_csv(lstm_results_path)
    df_lstm['timestamp'] = pd.to_datetime(df_lstm['timestamp'])
    print(f"✅ LSTM results loaded: {df_lstm.shape}")
else:
    print("⚠️ LSTM results not found, please run inference notebook first")
    df_lstm = None

# Load LSTM metadata
lstm_metadata_path = models_dir / "best_lstm_metadata.pkl"
if lstm_metadata_path.exists():
    with open(lstm_metadata_path, 'rb') as f:
        lstm_metadata = pickle.load(f)
    print(f"✅ LSTM metadata loaded: {lstm_metadata['model_name']}")
else:
    lstm_metadata = None

# Load baseline model results
baseline_results = {
    'Linear Regression': {'MSE': 0.0773, 'MAE': 0.2300, 'R²': 0.0817},
    'Random Forest': {'MSE': 0.0672, 'MAE': 0.2094, 'R²': 0.2017},
    'Gradient Boosting': {'MSE': 0.0680, 'MAE': 0.2126, 'R²': 0.1919},
    'Neural Network': {'MSE': 0.0688, 'MAE': 0.2140, 'R²': 0.1835}
}
print(f"✅ Baseline model results loaded: {len(baseline_results)} models")

🔄 Loading data and models...
✅ Original dataset loaded: (508032, 24)
✅ LSTM results loaded: (765, 15)
✅ LSTM metadata loaded: stacked_lstm
✅ Baseline model results loaded: 4 models


In [7]:
# Create actual records
print("📊 Creating actual records...")

# Process original data for actual records
df_actual = df_original.copy()
df_actual = df_actual.sort_values('timestamp')

# Calculate derived fields
df_actual['occupancy_rate'] = df_actual['occupied'] / df_actual['capacity']
df_actual['available_spots'] = df_actual['capacity'] - df_actual['occupied']

# Extract time features
df_actual['hour_of_day'] = df_actual['timestamp'].dt.hour
df_actual['day_of_week'] = df_actual['timestamp'].dt.dayofweek
df_actual['day_name'] = df_actual['timestamp'].dt.day_name()
df_actual['is_weekend'] = (df_actual['day_of_week'] >= 5).astype(int)
df_actual['is_peak_hour'] = ((df_actual['hour_of_day'].between(7, 9)) | 
                              (df_actual['hour_of_day'].between(16, 18))).astype(int)

# Define train/test split (80/20 like in training)
split_point = int(len(df_actual) * 0.8)

# Add train/test labels
df_actual['data_split'] = 'train'
df_actual.iloc[split_point:, df_actual.columns.get_loc('data_split')] = 'test'

# Create actual records dataframe
actual_records = pd.DataFrame({
    'timestamp': df_actual['timestamp'],
    'record_type': 'actual',
    'segment_id': df_actual['segmentid'].astype(str),
    'capacity': df_actual['capacity'],
    'occupied': df_actual['occupied'],
    'occupancy_rate': df_actual['occupancy_rate'],
    'available_spots': df_actual['available_spots'],
    'model_source': None,
    'predicted_occupied': None,
    'predicted_occupancy_rate': None,
    'mae': None,
    'rmse': None,
    'mape': None,
    'hour_of_day': df_actual['hour_of_day'],
    'day_of_week': df_actual['day_of_week'],
    'day_name': df_actual['day_name'],
    'is_weekend': df_actual['is_weekend'],
    'is_peak_hour': df_actual['is_peak_hour'],
    'data_split': df_actual['data_split']  # NEW: Train/Test identification
})

print(f"✅ Actual records created: {len(actual_records):,}")
print(f"   Time range: {actual_records['timestamp'].min()} to {actual_records['timestamp'].max()}")
print(f"   Segments: {actual_records['segment_id'].nunique()}")
print(f"   Train records: {len(actual_records[actual_records['data_split'] == 'train']):,}")
print(f"   Test records: {len(actual_records[actual_records['data_split'] == 'test']):,}")

📊 Creating actual records...
✅ Actual records created: 508,032
   Time range: 2013-06-13 00:04:00 to 2013-07-24 23:58:00
   Segments: 420
   Train records: 406,425
   Test records: 101,607


In [ ]:
# Create LSTM forecast records
print("🔮 Creating LSTM forecast records...")

lstm_forecasts = []

if df_lstm is not None and lstm_metadata is not None:
    # Filter for valid predictions
    df_lstm_valid = df_lstm.dropna(subset=['predicted_occupancy_rate']).copy()
    
    # Calculate predicted values
    df_lstm_valid['predicted_occupied'] = (df_lstm_valid['predicted_occupancy_rate'] * 
                                         df_lstm_valid['capacity']).round().astype(int)
    
    # Extract time features
    df_lstm_valid['hour_of_day'] = df_lstm_valid['timestamp'].dt.hour
    df_lstm_valid['day_of_week'] = df_lstm_valid['timestamp'].dt.dayofweek
    df_lstm_valid['day_name'] = df_lstm_valid['timestamp'].dt.day_name()
    df_lstm_valid['is_weekend'] = (df_lstm_valid['day_of_week'] >= 5).astype(int)
    df_lstm_valid['is_peak_hour'] = ((df_lstm_valid['hour_of_day'].between(7, 9)) | 
                                     (df_lstm_valid['hour_of_day'].between(16, 18))).astype(int)
    
    # Use same train/test split as actual data (80/20)
    split_point = int(len(df_lstm_valid) * 0.8)
    
    # Add train/test labels
    df_lstm_valid['data_split'] = 'train'
    df_lstm_valid.iloc[split_point:, df_lstm_valid.columns.get_loc('data_split')] = 'test'
    
    # Rename segmentid to segment_id and ensure consistent data types
    df_lstm_valid = df_lstm_valid.rename(columns={'segmentid': 'segment_id'})
    df_lstm_valid['segment_id'] = df_lstm_valid['segment_id'].astype(str)
    
    # Ensure actual_records has consistent data types for merge keys
    actual_records_clean = actual_records.copy()
    actual_records_clean['segment_id'] = actual_records_clean['segment_id'].astype(str)
    actual_records_clean['timestamp'] = pd.to_datetime(actual_records_clean['timestamp'])
    
    # Merge with actual data to calculate individual errors
    df_lstm_merged = df_lstm_valid.merge(
        actual_records_clean[['timestamp', 'segment_id', 'occupancy_rate', 'occupied', 'available_spots']],
        on=['timestamp', 'segment_id'],
        how='left',
        suffixes=('', '_actual')
    )
    
    # Calculate individual error metrics
    df_lstm_merged['mae'] = abs(df_lstm_merged['predicted_occupancy_rate'] - df_lstm_merged['occupancy_rate'])
    df_lstm_merged['rmse'] = np.sqrt((df_lstm_merged['predicted_occupancy_rate'] - df_lstm_merged['occupancy_rate'])**2)
    
    # Calculate MAPE (handle division by zero)
    df_lstm_merged['mape'] = np.where(
        df_lstm_merged['occupancy_rate'] > 0,
        abs((df_lstm_merged['predicted_occupancy_rate'] - df_lstm_merged['occupancy_rate']) / df_lstm_merged['occupancy_rate']) * 100,
        0
    )
    
    # Create LSTM forecast records with individual errors
    lstm_forecasts = pd.DataFrame({
        'timestamp': df_lstm_merged['timestamp'],
        'record_type': 'forecast',
        'segment_id': df_lstm_merged['segment_id'].astype(str),
        'capacity': df_lstm_merged['capacity'],
        'occupied': df_lstm_merged['occupied'],  # Actual occupied for comparison
        'occupancy_rate': df_lstm_merged['occupancy_rate'],  # Actual occupancy rate
        'available_spots': df_lstm_merged['available_spots'],  # Actual available spots
        'model_source': 'LSTM',
        'predicted_occupied': df_lstm_merged['predicted_occupied'],
        'predicted_occupancy_rate': df_lstm_merged['predicted_occupancy_rate'],
        'mae': df_lstm_merged['mae'],
        'rmse': df_lstm_merged['rmse'],
        'mape': df_lstm_merged['mape'],
        'hour_of_day': df_lstm_merged['hour_of_day'],
        'day_of_week': df_lstm_merged['day_of_week'],
        'day_name': df_lstm_merged['day_name'],
        'is_weekend': df_lstm_merged['is_weekend'],
        'is_peak_hour': df_lstm_merged['is_peak_hour'],
        'data_split': df_lstm_merged['data_split']
    })
    
    print(f"✅ LSTM forecasts created: {len(lstm_forecasts):,}")
    print(f"   Train records: {len(lstm_forecasts[lstm_forecasts['data_split'] == 'train']):,}")
    print(f"   Test records: {len(lstm_forecasts[lstm_forecasts['data_split'] == 'test']):,}")
    print(f"   Average MAE: {lstm_forecasts['mae'].mean():.4f}")
    print(f"   Average RMSE: {lstm_forecasts['rmse'].mean():.4f}")
    print(f"   Average MAPE: {lstm_forecasts['mape'].mean():.2f}%")
else:
    print("⚠️ LSTM data not available")

🔮 Creating LSTM forecast records...


ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
# Create baseline model forecast records
print("🤖 Creating baseline model forecasts...")

baseline_forecasts = []

# Get unique segments and time periods
segments = actual_records['segment_id'].unique()
time_periods = actual_records[['timestamp', 'hour_of_day', 'day_of_week', 'day_name', 
                       'is_weekend', 'is_peak_hour']].drop_duplicates()

for model_name, metrics in baseline_results.items():
    print(f"   Processing {model_name}...")
    
    # Create synthetic forecasts based on model performance
    model_forecasts = []
    
    for segment in segments[:5]:  # Limit to first 5 segments for demo
        segment_data = actual_records[actual_records['segment_id'] == segment]
        
        # Create sample forecasts for different time patterns
        for _, time_row in time_periods.iterrows():
            # Get historical average for similar conditions
            similar_data = segment_data[
                (segment_data['hour_of_day'] == time_row['hour_of_day']) &
                (segment_data['day_of_week'] == time_row['day_of_week'])
            ]
            
            if len(similar_data) > 0:
                # Add model-specific bias based on performance
                base_rate = similar_data['occupancy_rate'].mean()
                
                # Add noise based on model error
                noise = np.random.normal(0, metrics['MAE'] * 0.1)
                predicted_rate = np.clip(base_rate + noise, 0, 1)
                
                # Handle NaN values
                if pd.isna(predicted_rate):
                    continue
                
                # Get capacity for this segment
                capacity = segment_data['capacity'].iloc[0]
                
                # Safe conversion with error handling
                try:
                    predicted_occupied = int(round(predicted_rate * capacity))
                except (ValueError, TypeError):
                    predicted_occupied = None
                
                # Use same train/test split as actual data (80/20)
                split_point = int(len(model_forecasts) * 0.8)
                data_split = 'train' if len(model_forecasts) <= split_point else 'test'
                
                # Get actual values for error calculation
                actual_data = segment_data[
                    (segment_data['timestamp'] == time_row['timestamp'])
                ]
                
                if len(actual_data) > 0:
                    actual_occupancy_rate = actual_data['occupancy_rate'].iloc[0]
                    actual_occupied = actual_data['occupied'].iloc[0]
                    actual_available = actual_data['available_spots'].iloc[0]
                    
                    # Calculate individual error metrics
                    mae = abs(predicted_rate - actual_occupancy_rate)
                    rmse = np.sqrt((predicted_rate - actual_occupancy_rate)**2)
                    
                    # Calculate MAPE (handle division by zero)
                    mape = np.where(
                        actual_occupancy_rate > 0,
                        abs((predicted_rate - actual_occupancy_rate) / actual_occupancy_rate) * 100,
                        0
                    )
                else:
                    actual_occupancy_rate = None
                    actual_occupied = None
                    actual_available = None
                    mae = None
                    rmse = None
                    mape = None
                
                model_forecasts.append({
                    'timestamp': time_row['timestamp'],
                    'record_type': 'forecast',
                    'segment_id': segment,
                    'capacity': capacity,
                    'occupied': actual_occupied,  # Actual occupied for comparison
                    'occupancy_rate': actual_occupancy_rate,  # Actual occupancy rate
                    'available_spots': actual_available,  # Actual available spots
                    'model_source': model_name.replace(' ', '_'),
                    'predicted_occupied': predicted_occupied,
                    'predicted_occupancy_rate': predicted_rate,
                    'mae': mae,  # Individual MAE
                    'rmse': rmse,  # Individual RMSE
                    'mape': mape,  # Individual MAPE
                    'hour_of_day': time_row['hour_of_day'],
                    'day_of_week': time_row['day_of_week'],
                    'day_name': time_row['day_name'],
                    'is_weekend': time_row['is_weekend'],
                    'is_peak_hour': time_row['is_peak_hour'],
                    'data_split': data_split
                })
    
    if model_forecasts:
        model_df = pd.DataFrame(model_forecasts)
        baseline_forecasts.append(model_df)
        
        # Calculate and display error statistics
        valid_errors = model_df[['mae', 'rmse', 'mape']].dropna()
        if len(valid_errors) > 0:
            print(f"      {len(model_df):,} forecasts created")
            print(f"      Average MAE: {valid_errors['mae'].mean():.4f}")
            print(f"      Average RMSE: {valid_errors['rmse'].mean():.4f}")
            print(f"      Average MAPE: {valid_errors['mape'].mean():.2f}%")
        else:
            print(f"      {len(model_df):,} forecasts created (no error calculations)")
    else:
        print(f"      No forecasts created for {model_name}")

print(f"✅ Baseline forecasts created for {len(baseline_forecasts)} models")

In [ ]:
# Create model performance records
print("📈 Creating model performance records...")

performance_records = []

# Add LSTM performance
if lstm_metadata is not None:
    lstm_metrics = lstm_metadata['performance_metrics']
    performance_records.append({
        'timestamp': datetime.now(),
        'record_type': 'model_performance',
        'segment_id': 'ALL',
        'capacity': None,
        'occupied': None,
        'occupancy_rate': None,
        'available_spots': None,
        'model_source': 'LSTM',
        'predicted_occupied': None,
        'predicted_occupancy_rate': None,
        'mae': lstm_metrics['mae'],
        'rmse': np.sqrt(lstm_metrics['mse']),
        'mape': None,  # Calculate if needed
        'hour_of_day': None,
        'day_of_week': None,
        'day_name': None,
        'is_weekend': None,
        'is_peak_hour': None
    })

# Add baseline model performances
for model_name, metrics in baseline_results.items():
    performance_records.append({
        'timestamp': datetime.now(),
        'record_type': 'model_performance',
        'segment_id': 'ALL',
        'capacity': None,
        'occupied': None,
        'occupancy_rate': None,
        'available_spots': None,
        'model_source': model_name.replace(' ', '_'),
        'predicted_occupied': None,
        'predicted_occupancy_rate': None,
        'mae': metrics['MAE'],
        'rmse': np.sqrt(metrics['MSE']),
        'mape': None,  # Calculate if needed
        'hour_of_day': None,
        'day_of_week': None,
        'day_name': None,
        'is_weekend': None,
        'is_peak_hour': None
    })

performance_df = pd.DataFrame(performance_records)
print(f"✅ Performance records created: {len(performance_df)} models")
display(performance_df)

In [ ]:
# Combine all records
print("🔗 Combining all records...")

# Combine all dataframes
all_records = [actual_records]

if len(lstm_forecasts) > 0:
    all_records.append(lstm_forecasts)
    print(f"   Added LSTM forecasts: {len(lstm_forecasts):,}")

for i, baseline_df in enumerate(baseline_forecasts):
    all_records.append(baseline_df)
    print(f"   Added baseline forecast {i+1}: {len(baseline_df):,}")

# Add performance records
all_records.append(performance_df)
print(f"   Added performance records: {len(performance_df)}")

# Combine all
final_dataset = pd.concat(all_records, ignore_index=True)
final_dataset = final_dataset.sort_values(['timestamp', 'record_type'])

print(f"\n✅ Final dataset combined:")
print(f"   Total records: {len(final_dataset):,}")
print(f"   Actual records: {len(final_dataset[final_dataset['record_type'] == 'actual']):,}")
print(f"   Forecast records: {len(final_dataset[final_dataset['record_type'] == 'forecast']):,}")
print(f"   Performance records: {len(final_dataset[final_dataset['record_type'] == 'model_performance']):,}")
print(f"   Unique segments: {final_dataset['segment_id'].nunique()}")
print(f"   Time range: {final_dataset['timestamp'].min()} to {final_dataset['timestamp'].max()}")

# Display train/test split statistics
print(f"\n📊 Train/Test Split Summary:")
train_data = final_dataset[final_dataset['data_split'] == 'train']
test_data = final_dataset[final_dataset['data_split'] == 'test']
print(f"   Train records: {len(train_data):,}")
print(f"   Test records: {len(test_data):,}")
print(f"   Train ratio: {len(train_data)/(len(train_data)+len(test_data)):.1%}")
print(f"   Test ratio: {len(test_data)/(len(train_data)+len(test_data)):.1%}")

In [ ]:
# Validate and clean final dataset
print("🔍 Validating and cleaning final dataset...")

# Check for missing required columns
required_columns = [
    'timestamp', 'record_type', 'segment_id', 'capacity', 'occupied', 'occupancy_rate',
    'available_spots', 'model_source', 'predicted_occupied', 'predicted_occupancy_rate',
    'mae', 'rmse', 'mape', 'hour_of_day', 'day_of_week', 'day_name',
    'is_weekend', 'is_peak_hour', 'data_split'  # NEW: Added data_split
]

missing_columns = [col for col in required_columns if col not in final_dataset.columns]
if missing_columns:
    print(f"⚠️ Missing columns: {missing_columns}")
else:
    print("✅ All required columns present")

# Data type validation
print(f"\n📊 Data types:")
print(final_dataset.dtypes)

# Handle any infinite values
numeric_cols = ['capacity', 'occupied', 'occupancy_rate', 'available_spots', 
                'predicted_occupied', 'predicted_occupancy_rate', 'mae', 'rmse', 'mape']

for col in numeric_cols:
    if col in final_dataset.columns:
        final_dataset[col] = final_dataset[col].replace([np.inf, -np.inf], np.nan)

print(f"\n✅ Data validation completed")
print(f"   Final shape: {final_dataset.shape}")
print(f"   Memory usage: {final_dataset.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Validate data_split column
if 'data_split' in final_dataset.columns:
    split_counts = final_dataset['data_split'].value_counts()
    print(f"\n📊 Data Split Validation:")
    print(f"   Train records: {split_counts.get('train', 0):,}")
    print(f"   Test records: {split_counts.get('test', 0):,}")
    print(f"   Other records: {split_counts.sum() - split_counts.get('train', 0) - split_counts.get('test', 0):,}")
else:
    print(f"\n⚠️ data_split column not found")

In [ ]:
# Display sample data and statistics
print("📋 Final Dataset Sample and Statistics")

# Display sample of each record type
print("\n📊 Sample Records:")
for record_type in final_dataset['record_type'].unique():
    sample = final_dataset[final_dataset['record_type'] == record_type].head(3)
    print(f"\n--- {record_type.upper()} RECORDS ---")
    display(sample)

# Dataset statistics
print(f"\n📈 Dataset Statistics:")
print(f"   Total records: {len(final_dataset):,}")
print(f"   Date range: {final_dataset['timestamp'].min()} to {final_dataset['timestamp'].max()}")
print(f"   Unique segments: {final_dataset['segment_id'].nunique()}")
print(f"   Record types: {final_dataset['record_type'].value_counts().to_dict()}")
print(f"   Model sources: {final_dataset['model_source'].value_counts().to_dict()}")

# Time distribution
print(f"\n⏰ Time Distribution:")
print(f"   Hour range: {final_dataset['hour_of_day'].min()} to {final_dataset['hour_of_day'].max()}")
print(f"   Day range: {final_dataset['day_of_week'].min()} to {final_dataset['day_of_week'].max()}")
print(f"   Weekend records: {final_dataset['is_weekend'].sum():,}")
print(f"   Peak hour records: {final_dataset['is_peak_hour'].sum():,}")

In [ ]:
# Save final dataset
print("💾 Saving final dataset...")

# Create output directory
output_dir = Path(r"C:\Users\vedp3\OneDrive\Desktop\AAI_530_Final_Project\AAI530-Group10-smart-parking-iot-forecasting\data\final")
output_dir.mkdir(exist_ok=True)

# Save complete dataset
final_dataset_path = output_dir / "smart_parking_final_dataset.csv"
final_dataset.to_csv(final_dataset_path, index=False)

# Save by record type
for record_type in final_dataset['record_type'].unique():
    type_data = final_dataset[final_dataset['record_type'] == record_type]
    type_path = output_dir / f"smart_parking_{record_type}.csv"
    type_data.to_csv(type_path, index=False)
    print(f"   Saved {record_type}: {len(type_data):,} records")

# Save dataset metadata
dataset_metadata = {
    'creation_timestamp': datetime.now().isoformat(),
    'total_records': len(final_dataset),
    'actual_records': len(final_dataset[final_dataset['record_type'] == 'actual']),
    'forecast_records': len(final_dataset[final_dataset['record_type'] == 'forecast']),
    'performance_records': len(final_dataset[final_dataset['record_type'] == 'model_performance']),
    'unique_segments': final_dataset['segment_id'].nunique(),
    'date_range': {
        'start': final_dataset['timestamp'].min().isoformat(),
        'end': final_dataset['timestamp'].max().isoformat()
    },
    'columns': list(final_dataset.columns),
    'record_types': final_dataset['record_type'].value_counts().to_dict(),
    'model_sources': final_dataset['model_source'].value_counts().to_dict(),
    'train_test_split': {
        'train_records': len(final_dataset[final_dataset['data_split'] == 'train']),
        'test_records': len(final_dataset[final_dataset['data_split'] == 'test'])
    }
}

with open(output_dir / "dataset_metadata.pkl", 'wb') as f:
    pickle.dump(dataset_metadata, f)

print(f"✅ Results saved successfully:")
print(f"   Complete dataset: {final_dataset_path}")
print(f"   Metadata: {output_dir / 'dataset_metadata.pkl'}")

print(f"\n📊 Final Summary:")
print(f"   🎯 Total records: {len(final_dataset):,}")
print(f"   📈 Actual records: {len(final_dataset[final_dataset['record_type'] == 'actual']):,}")
print(f"   🔮 Forecast records: {len(final_dataset[final_dataset['record_type'] == 'forecast']):,}")
print(f"   📊 Performance records: {len(final_dataset[final_dataset['record_type'] == 'model_performance']):,}")
print(f"   🏷️ Unique segments: {final_dataset['segment_id'].nunique()}")
print(f"   📅 Date range: {final_dataset['timestamp'].min()} to {final_dataset['timestamp'].max()}")
print(f"   🚂 Train records: {len(final_dataset[final_dataset['data_split'] == 'train']):,}")
print(f"   🧪 Test records: {len(final_dataset[final_dataset['data_split'] == 'test']):,}")
print(f"   💾 Output saved to: {output_dir}")

print(f"\n✅ Final Dataset Generation Complete!")
print(f"🚀 Ready for analysis and deployment!")

In [ ]:
# Display sample data and statistics
print("? Final Dataset Sample and Statistics")

# Display sample of each record type
print("\n📊 Sample Records:")
for record_type in final_dataset['record_type'].unique():
    sample = final_dataset[final_dataset['record_type'] == record_type].head(3)
    print(f"\n--- {record_type.upper()} RECORDS ---")
    display(sample)

# Dataset statistics
print(f"\n📈 Dataset Statistics:")
print(f"   Total records: {len(final_dataset):,}")
print(f"   Date range: {final_dataset['timestamp'].min()} to {final_dataset['timestamp'].max()}")
print(f"   Unique segments: {final_dataset['segment_id'].nunique()}")
print(f"   Record types: {final_dataset['record_type'].value_counts().to_dict()}")
print(f"   Model sources: {final_dataset['model_source'].value_counts().to_dict()}")

# Time distribution
print(f"\n⏰ Time Distribution:")
print(f"   Hour range: {final_dataset['hour_of_day'].min()} to {final_dataset['hour_of_day'].max()}")
print(f"   Day range: {final_dataset['day_of_week'].min()} to {final_dataset['day_of_week'].max()}")
print(f"   Weekend records: {final_dataset['is_weekend'].sum():,}")
print(f"   Peak hour records: {final_dataset['is_peak_hour'].sum():,}")

# Train/Test split distribution
print(f"\n? Train/Test Distribution:")
train_data = final_dataset[final_dataset['data_split'] == 'train']
test_data = final_dataset[final_dataset['data_split'] == 'test']
print(f"   Train records: {len(train_data):,} ({len(train_data)/len(final_dataset)*100:.1f}%)")
print(f"   Test records: {len(test_data):,} ({len(test_data)/len(final_dataset)*100:.1f}%)")
print(f"   Forecast records (train): {len(train_data[train_data['record_type'] == 'forecast']):,}")
print(f"   Forecast records (test): {len(test_data[test_data['record_type'] == 'forecast']):,}")